In [106]:
import pandas as pd
import os
import urllib.request
import cv2
import requests
import base64
import re
import shutil

from os.path import exists
from html2image import Html2Image


class CardGenerator:
    #Hyperparameters for the Generator
    #IO-Paths
    PATH_ARTWORK_DATA = './artwork_data/' #TODO: This is weird because the relative references to the structure files of the card can't be found if the files are created in another folder
    PATH_SVG_OUTPUT = './generated_cards/'
    PATH_TEMPLATE_FOLDER = './card_template/'
    PATH_IMAGES_FOLDER = './generated_cards_images/'
    
    PATH_CARD_TEMPLATE = PATH_TEMPLATE_FOLDER + 'card_template.svg'
    PATH_EXCEL_SHEET = './cards.xlsx'
    PATH_WEBSITE_RENDER = PATH_SVG_OUTPUT + 'website_render.html'
    PATH_TEMPORARY_SVG = PATH_SVG_OUTPUT + 'TEMPORARY_CARD.svg'
    
    #highest amount of characters in a line in the effect-textox
    MAX_CHAR_PER_LINE = 25
    MAX_LINES_PER_CARD = 6
    MAX_LENGTH_OF_NAME_WITHOUT_STRETCH = 15
    CARD_WIDTH_PX = 900
    CARD_HEIGHT_PX = 1260

    #even if the corresponding artwork file already exists, download the picture from the link and overwrite the current picture
    ENFORCE_ART_DOWNLOAD = False
    
    #The name of the game (used as a prefix)
    GAME_NAME = 'Beyond'
    
    def __init__(self):
        print('Check and generate folders...')    
        
        if not os.path.exists(self.PATH_ARTWORK_DATA):
            print('Created ' + self.PATH_ARTWORK_DATA)
            os.makedirs(self.PATH_ARTWORK_DATA)
            
        if not os.path.exists(self.PATH_IMAGES_FOLDER):
            print('Created ' + self.PATH_IMAGES_FOLDER)
            os.makedirs(self.PATH_IMAGES_FOLDER)
            
        if not os.path.exists(self.PATH_SVG_OUTPUT):
            print('Created ' + self.PATH_SVG_OUTPUT)
            os.makedirs(self.PATH_SVG_OUTPUT)
            
        if not os.path.exists(self.PATH_TEMPLATE_FOLDER):
            os.makedirs(self.PATH_TEMPLATE_FOLDER)
            raise Exception('The folder ', PATH_TEMPLATE_FOLDER, ' does not exist! Please make sure that the folder and all the template-data/images are included!')
        
        
        print('Trying to read card database from ' + self.PATH_EXCEL_SHEET + '...')
        #Create SVG files based on Excel sheet
        #Excel sheet #1 = Mech-Karten (mit Header)
        #Excel sheet #2 = Upgrade-Shop (X=5 Zeilen pro Karte (inkl. 1 Header/Namen-Zeile), danach ein Blank-Space. Darunter weitere Karten)
        #Excel sheet #1 = Sieges-Karten (mit Header)
        #Excel sheet #1 = Aktions-Karten (mit Header)

        #create pandas dataframe and read all sheets into dictionary with given keys
        self.database = pd.read_excel(
             self.PATH_EXCEL_SHEET,
             dtype=str,
             sheet_name=None,
             engine='openpyxl'
        )
        
    def generate_cards(self, excelSheetName):
        currentDatabase = self.database[excelSheetName]
        header = currentDatabase.columns.values
        
        print('Reading data from sheet "', excelSheetName, '" with the following columns: ' + str(header))
        
        for cardNo in range(currentDatabase.shape[0]):
            #convert the current card into a stat dictionary
            statDictionary = self.createCardDictionary(currentDatabase, cardNo, header)           

            #DEL: print('Read into dictionary: ' + str(statDictionary))
        
            #this is not a valid card -> next!
            if statDictionary is None:
                continue
            
            #TODO: FROM HERE ON -> OUTSOURCE THIS ENTIRE CODE INTO A METHOD THAT EXPANDS THE DICTIONARY WITH REQUIRED STATS
        
            imageFilename = 'Artwork_' + statDictionary['NAME'] + '.png'
            desiredImagePath = self.PATH_ARTWORK_DATA + imageFilename

            if ('ARTWORKURL' in statDictionary and (imageFilename not in os.listdir(self.PATH_ARTWORK_DATA)) or self.ENFORCE_ART_DOWNLOAD):
                #file doesnt exist or is forced to be redownloaded

                artworkURL = statDictionary['ARTWORKURL']
                desiredImagePath = self.PATH_SVG_OUTPUT + statDictionary['NAME'] + '.png'
                
                if self.isValidString(artworkURL):
                    #download artwork and save it
                    response = requests.get(artworkURL)
                    file = open(desiredImagePath, "wb")
                    file.write(response.content)
                    file.close()
                                 
                    #write back the downloaded artwork to the dictionary
                    statDictionary['ARTWORKURL'] = desiredImagePath
                else:
                    artworkFileExists = False
            else:
                artworkFileExists = False

            #TODO: Fix this ugly pre-processing to make the line-break work
            statDictionary['EFFECT'] = statDictionary['EFFECT'].replace('..',' .. ')
                
            #Split the cardtext into multiple lines in the effect-textbox
            if 'EFFECT' in statDictionary:
                #add additional entries for the dictionary (Effekt0,Effekt1,...)
                currentEffectText = statDictionary['EFFECT']
                currentEffectTextSplit = currentEffectText.split()

                newEffectText = []
                newEffectLine = ''
                for singleWord in currentEffectTextSplit:
                    #line break: Instantly jump to next line 
                    if singleWord == '..':
                        newEffectText.append(newEffectLine)
                        newEffectLine = ''
                    else:
                        newEffectLine = newEffectLine + ' ' + singleWord

                    #if the delimiter for the current line is exceeded
                    if len(newEffectLine) > self.MAX_CHAR_PER_LINE:
                        #save current line, reset, continue with next words
                        newEffectText.append(newEffectLine)
                        newEffectLine = ''   
                                                        
                #we still have words remaining? -> add them at the end
                if len(newEffectLine) > 0:
                    newEffectText.append(newEffectLine)

                #save back all new effect lines into the dictionary, split for each line
                if len(newEffectText) >= self.MAX_LINES_PER_CARD:
                    print('WARNING: Card #', str(cardNo), ' exceeds the boundary of ', str(self.MAX_LINES_PER_CARD), ' lines. The text is too long:', newEffectText)
                    
                    
                for i in range(len(newEffectText)):
                    statDictionary['EFFECT' + str(i+1)] = newEffectText[i]
                #fill the rest of the entries with 0
                for i in range(len(newEffectText), self.MAX_LINES_PER_CARD):
                    statDictionary['EFFECT' + str(i+1)] = ''
                
            #TODO: For extra long text -> work with SVG token queezer
                                                        
            
            
            #Place the crystals in the right place with the right number on top
              
            #TODO: MAKE THIS BEAUTIFUL FOR RASHID <3
            #TODO: Ich kriege Augenkrebs bei diesem Code
            #COST1 = Ruby, COST2 = Sapphire, COST3=Topaz
            statDictionary['COST1'] = str(statDictionary['COST'].count('{R}'))
            statDictionary['COST2'] = str(statDictionary['COST'].count('{B}'))
            statDictionary['COST3'] = str(statDictionary['COST'].count('{Y}'))
            
            statDictionary['CRYSTAL1'] = 'Ruby'
            statDictionary['CRYSTAL2'] = 'Sapphire'
            statDictionary['CRYSTAL3'] = 'Topaz'
            
            if(statDictionary['COST'].count('{R}') == 0):
                statDictionary['CRYSTAL1_OPACITY'] = '0.3'
            else:
                statDictionary['CRYSTAL1_OPACITY'] = '1'
                
            if(statDictionary['COST'].count('{B}') == 0):
                statDictionary['CRYSTAL2_OPACITY'] = '0.3'
            else:
                statDictionary['CRYSTAL2_OPACITY'] = '1'
                
            if(statDictionary['COST'].count('{Y}') == 0):
                statDictionary['CRYSTAL3_OPACITY'] = '0.3'
            else:
                statDictionary['CRYSTAL3_OPACITY'] = '1'
                
            #Include spacing/shrinking of the title if it's too long
            if len(statDictionary['NAME']) > self.MAX_LENGTH_OF_NAME_WITHOUT_STRETCH:
                statDictionary['NAME_STRETCH_PERCENTAGE'] = '54'
            else:
                statDictionary['NAME_STRETCH_PERCENTAGE'] = '0'
                
            #Generate Card-Type and Subtype
            additionalSubtype = ''
            if self.isValidString(statDictionary['SUBTYPE']):
                additionalSubtype = ' - ' + statDictionary['SUBTYPE']
            
            statDictionary['CARDTYPE'] = statDictionary['CARDTYPE'] + additionalSubtype
                                                   
            #TODO: Very weird: Convert all relative paths in the SVG to global absollute paths      
            statDictionary['.'] = os.getcwd()
            
            #create the final svg output and move to folder
            newSvgFile = self.PATH_SVG_OUTPUT + self.GAME_NAME + '_' + statDictionary['NAME'] + '.svg'
            shutil.copyfile(self.PATH_CARD_TEMPLATE, newSvgFile)
            
            #DEL: Is this line necessary? Python OMEGALUL
            templateText = ''
            
            #read the template and transport data
            with open(newSvgFile,'r',encoding='utf-8') as cardSvgFile:   
                templateText = cardSvgFile.read()
            
                for singleData in statDictionary:
                    dataWildcard = '#' + singleData + '#'
                    templateText = templateText.replace(dataWildcard, statDictionary[singleData])
                
                
            #TODO: Transform all valid image paths from the SVG into base64 and write back
            #iterate over all possible image paths in current templateText
            linkPattern = re.compile(r'[A-Z]:\\(?:.*?\\)*[^\"]+\.[^\"\s]+')
            
            for link in re.findall(linkPattern, templateText):
                if exists(link):
                    #get image dimensions
                    imageDimensions = cv2.imread(link).shape
                    
                    #convert to base64 and write back
                    with open(link, 'rb') as imageFile:
                        base64Image = 'data:image/png;base64,' + base64.b64encode(imageFile.read()).decode() + '" height="' + str(imageDimensions[0]) + '" width="' + str(imageDimensions[1])
                    #write back the base64 picture to the SVG file and replace
                    templateText = templateText.replace(link, base64Image)
            
            
            #write the card data back
            with open(newSvgFile,'w',encoding='utf-8') as cardSvgFile: 
                cardSvgFile.write(templateText)

    def createCardDictionary(self, rawData, cardNo, headerFormat):
        returnDict = {}

        #TODO: This is quite awkward
        #Sanity-check: We expect that all cards have an unique name identifier. If a name is not given, we will return an error
        if self.isValidString(rawData['Name'][cardNo]) is False:
            return None
        
        for columnName in headerFormat:
            data = str(rawData[columnName][cardNo])
                       
            #if text is equal to 'nan' then don't print it
            if (columnName.startswith('Unnamed')) is False:
                #normalize header name for easier data fetching
                returnDict[columnName.replace(' ','').upper()] = data
            
        return returnDict
    
    def isValidString(self, string):
        return ((str(string) is None or str(string) == 'nan' or len(str(string))==0) is False)
    
    def svg2png(self):
        hti = Html2Image(output_path=self.PATH_IMAGES_FOLDER)
        
        allSvgFiles = os.listdir(self.PATH_SVG_OUTPUT)
        
        for singleFile in allSvgFiles:
            fullSvgPath = self.PATH_SVG_OUTPUT + singleFile
            
            #copy current SVG to temporary SVG for processing purposes
            shutil.copyfile(fullSvgPath, self.PATH_TEMPORARY_SVG)

            #TODO: Make this prettier
            pngFileName = singleFile.replace('.svg', '.png')
            #Render the png from the html page, that by itself includes the info from the temporary SVG file
            
            #TODO: Fix this so this generates from the HTML file (currently doesn't load the image if trying to create from HTML)
            hti.screenshot(other_file = fullSvgPath, save_as = pngFileName, size = (self.CARD_WIDTH_PX, self.CARD_HEIGHT_PX))

            print('Generated card "', pngFileName, '".')

In [107]:
generator = CardGenerator()
generator.generate_cards('0.9 Alpha (Core Set = CS)')
generator.svg2png()

Check and generate folders...
Trying to read card database from ./cards.xlsx...
Reading data from sheet " 0.9 Alpha (Core Set = CS) " with the following columns: ['Name' 'Realm' 'Level' 'Cost' 'Card Type' 'Effect' 'Attack' 'HP'
 'Subtype' 'Rarity' 'Nummer' 'Artwork URL' 'Unnamed: 12' 'Unnamed: 13'
 'Unnamed: 14' 'Unnamed: 15' 'Unnamed: 16' 'Unnamed: 17']
Generated card " Beyond_Ambitious Enchantress.png ".
Generated card " Beyond_Aspiring Blacksmith.png ".
Generated card " Beyond_Avatar of Fujin.png ".
Generated card " Beyond_Awaken the Phoenix.png ".
Generated card " Beyond_Baby Iamassu.png ".
Generated card " Beyond_Blightcoven Monstrosity.png ".
Generated card " Beyond_Blinding Light.png ".
Generated card " Beyond_Bloodfleet Raider.png ".
Generated card " Beyond_Brainwash.png ".
Generated card " Beyond_Bronze Shard.png ".
Generated card " Beyond_Cryosaurus.png ".
Generated card " Beyond_Darkspoon.png ".
Generated card " Beyond_Disturbed Lindwurm.png ".
Generated card " Beyond_Domest

SameFileError: './generated_cards/TEMPORARY_CARD.svg' and './generated_cards/TEMPORARY_CARD.svg' are the same file

['C:\\Users\\Jonas Meise\\Documents\\GitHub\\BeyondCardGame\\test.png']